## Teste de PostgrSQL



In [2]:
import psycopg2

conn = psycopg2.connect(
    dbname="detectid",
    user="postgres",
    password="Goncalo123",
    host="localhost"
)

cur = conn.cursor()
cur.execute("SELECT * FROM cdmdatabaseschema.person_details LIMIT 5;")
print(cur.fetchall())

cur.close()
conn.close()

[]


### Carregar Ficheiro

In [3]:
import pandas as pd


file_path = "detectid.csv"
df = pd.read_csv(file_path)


df.head()

,Pessoa,Tempo,Dia de Medição,Hora de Medição,Data de Nascimento,Genero,Serviço,NIVEL DE CONSCIÊNCIA,FREQUÊNCIA CARDIACA,TA Sistólica,...,TEMPERATURA,SpO2,NECESSIDADE DE O2,DOR,DESCOMPENSAÇÃO,Ativação Médico,Aumento da Vigilância,Via Área Ameaçada,Evento,Area\r\nMonotorizada
0,Pessoa 1,0.0,11/11/2023,11:00,04/08/1965,Masculino,1,15,103.0,109.0,...,37.1,94.0,1.0,0.0,0.0,0.0,0.0,0.0,0,NaN
1,Pessoa 1,125.5,16/11/2023,18:30,04/08/1965,Masculino,2,15,99.0,110.0,...,37.1,96.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
2,Pessoa 1,127.5,16/11/2023,20:30,04/08/1965,Masculino,2,15,78.0,102.0,...,NaN,NaN,0.0,0.0,1.0,1.0,1.0,0.0,1,1.0
3,Pessoa 1,133.0,17/11/2023,02:00,04/08/1965,Masculino,2,15,108.0,87.0,...,36.8,96.0,0.0,0.0,1.0,0.0,1.0,0.0,1,1.0
4,Pessoa 1,138.5,17/11/2023,7:30,04/08/1965,Masculino,2,15,96.0,94.0,...,37.1,NaN,0.0,0.0,1.0,0.0,1.0,0.0,1,1.0


### Tratemento dos Dados

Aqui chatgpt para tratamento dos dados -> Rever isto tudo

In [4]:
# Tempo tem que ser calculado logo nao entra na base de dados
# Evento foi uma experiencia, tambem nao entra
df_cleaned = df.drop(columns=["Tempo", "Evento"])

df_cleaned.rename(columns=lambda x: x.strip().replace("\n", " "), inplace=True)

# Valores Omissos preenchidos com média
numeric_cols = df_cleaned.select_dtypes(include='number').columns
df_cleaned[numeric_cols] = df_cleaned[numeric_cols].fillna(df_cleaned[numeric_cols].mean())

# Converter para numérico (caso haja strings ou espaços)
df_cleaned["NIVEL DE CONSCIÊNCIA"] = pd.to_numeric(df_cleaned["NIVEL DE CONSCIÊNCIA"], errors='coerce')

# Calcular a média ignorando os NaNs e arredondar para inteiro
media_nivel = int(df_cleaned["NIVEL DE CONSCIÊNCIA"].mean().round())

# Preencher os NaNs com a média
df_cleaned["NIVEL DE CONSCIÊNCIA"].fillna(media_nivel, inplace=True)

# Converter a coluna para datetime (assumindo formato MM/DD/YYYY)
df_cleaned["Data de Nascimento"] = pd.to_datetime(df_cleaned["Data de Nascimento"], format="%m/%d/%Y", errors="coerce")

# Criar colunas separadas para ano, mês e dia
df_cleaned["Ano de Nascimento"] = df_cleaned["Data de Nascimento"].dt.year
df_cleaned["Mes de Nascimento"] = df_cleaned["Data de Nascimento"].dt.month
df_cleaned["Dia de Nascimento"] = df_cleaned["Data de Nascimento"].dt.day

# Garantir que 'Dia' e 'Hora' são string
df_cleaned["Dia de Medição"] = df_cleaned["Dia de Medição"].astype(str)
df_cleaned["Hora de Medição"] = df_cleaned["Hora de Medição"].astype(str)

# Juntar as colunas 'Dia' e 'Hora' e converter para datetime com formato automático
df_cleaned["datetime"] = pd.to_datetime(df_cleaned["Dia de Medição"] + " " + df_cleaned["Hora de Medição"], dayfirst=True, errors="coerce")

df_cleaned["person_id"] = df_cleaned["Pessoa"].str.extract(r"(\d+)").astype(int)

# Ver exemplo de dados limpos e prontos para inserção
print(df_cleaned.to_string())


         Pessoa Dia de Medição Hora de Medição Data de Nascimento      Genero  Serviço  NIVEL DE CONSCIÊNCIA  FREQUÊNCIA CARDIACA  TA Sistólica  TA Diastólica  TEMPERATURA        SpO2  NECESSIDADE DE O2       DOR  DESCOMPENSAÇÃO  Ativação Médico  Aumento da Vigilância  Via Área Ameaçada  Area\r Monotorizada  Ano de Nascimento  Mes de Nascimento  Dia de Nascimento            datetime  person_id
0      Pessoa 1     11/11/2023           11:00         1965-04-08   Masculino        1                  15.0           103.000000    109.000000      61.000000    37.100000   94.000000           1.000000  0.000000        0.000000         0.000000               0.000000               0.00             0.088889               1965                  4                  8 2023-11-11 11:00:00          1
1      Pessoa 1     16/11/2023           18:30         1965-04-08   Masculino        2                  15.0            99.000000    110.000000      74.000000    37.100000   96.000000           0.000000  0.

C:\Users\Goncalo\AppData\Local\Temp\ipykernel_23076\3459433511.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cleaned["NIVEL DE CONSCIÊNCIA"].fillna(media_nivel, inplace=True)


### Inserção dos dados PERSON

In [5]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values

# Conectar à base de dados
conn = psycopg2.connect(
    dbname="detectid",
    user="postgres",
    password="Goncalo123",
    host="localhost",
    port="5432"
)

cur = conn.cursor()

rows = []
person_id = 0
for _, row in df_cleaned.iterrows():
        if person_id < row["person_id"]:
            person_id = row["person_id"]
            genero = 1 if row['Genero'] == 'Masculino' else 0
            rows.append((
                person_id,
                genero,
                row["Ano de Nascimento"],
                row["Mes de Nascimento"],
                row["Dia de Nascimento"],
                "12345555"
            ))

query = """
INSERT INTO cdmDatabaseSchema.PERSON (
    person_id, gender_concept_id, year_of_birth, month_of_birth, day_of_birth, person_source_value
) VALUES %s
"""

execute_values(cur, query, rows)
conn.commit()

print("Dados PERSON inseridos com sucesso!")

cur.close()
conn.close()


Dados PERSON inseridos com sucesso!


### Inserção dos Dados MEASUREMENT

In [6]:
# Conectar à base de dados
conn = psycopg2.connect(
    dbname="detectid",
    user="postgres",
    password="Goncalo123",
    host="localhost",
    port="5432"
)

cur = conn.cursor()

# Exemplo de inserção na tabela MEASUREMENT
measurement_concepts = {
    "TA Sistólica": 1,
    "TA Diastólica": 2,
    "TEMPERATURA": 3,
    "SpO2": 4,
    "DOR": 5,
    "NECESSIDADE DE O2": 6,
    "FREQUÊNCIA CARDIACA": 7,
    "NIVEL DE CONSCIÊNCIA": 8
}

measurement_id = 1
rows = []
for _, row in df_cleaned.iterrows():
    for field, concept_id in measurement_concepts.items():
        rows.append((
            measurement_id,
            int(row["person_id"]),
            concept_id,
            row[field],
            row["datetime"]
        ))
        measurement_id += 1

query = """
INSERT INTO cdmDatabaseSchema.MEASUREMENT (
    measurement_id, person_id, measurement_concept_id, value_as_number, measurement_datetime
) VALUES %s
"""


execute_values(cur, query, rows)
conn.commit()

print("Dados MEASUREMENT inseridos com sucesso!")
cur.close()
conn.close()


Dados MEASUREMENT inseridos com sucesso!


In [7]:
print(rows)

[(1, 1, 1, 109.0, Timestamp('2023-11-11 11:00:00')), (2, 1, 2, 61.0, Timestamp('2023-11-11 11:00:00')), (3, 1, 3, 37.1, Timestamp('2023-11-11 11:00:00')), (4, 1, 4, 94.0, Timestamp('2023-11-11 11:00:00')), (5, 1, 5, 0.0, Timestamp('2023-11-11 11:00:00')), (6, 1, 6, 1.0, Timestamp('2023-11-11 11:00:00')), (7, 1, 7, 103.0, Timestamp('2023-11-11 11:00:00')), (8, 1, 8, 15.0, Timestamp('2023-11-11 11:00:00')), (9, 1, 1, 110.0, Timestamp('2023-11-16 18:30:00')), (10, 1, 2, 74.0, Timestamp('2023-11-16 18:30:00')), (11, 1, 3, 37.1, Timestamp('2023-11-16 18:30:00')), (12, 1, 4, 96.0, Timestamp('2023-11-16 18:30:00')), (13, 1, 5, 0.0, Timestamp('2023-11-16 18:30:00')), (14, 1, 6, 0.0, Timestamp('2023-11-16 18:30:00')), (15, 1, 7, 99.0, Timestamp('2023-11-16 18:30:00')), (16, 1, 8, 15.0, Timestamp('2023-11-16 18:30:00')), (17, 1, 1, 102.0, Timestamp('2023-11-16 20:30:00')), (18, 1, 2, 78.0, Timestamp('2023-11-16 20:30:00')), (19, 1, 3, 36.580681818181816, Timestamp('2023-11-16 20:30:00')), (20, 1

### Inserção dos Dados CARE_SITE
1 Urgência 2 Cuidados 3 UCI

In [8]:
# Conectar à base de dados
conn = psycopg2.connect(
    dbname="detectid",
    user="postgres",
    password="Goncalo123",
    host="localhost",
    port="5432"
)

cur = conn.cursor()

query = """
INSERT INTO cdmdatabaseschema.CARE_SITE (care_site_id)
VALUES %s
"""
rows = [(1,),(2,),(3,)]
execute_values(cur, query, rows)
conn.commit()

print("Dados CARE_SITE inseridos com sucesso!")
cur.close()
conn.close()

Dados CARE_SITE inseridos com sucesso!


### Inserção dos Dados CONDITION_OCCURENCE
Diagnósticos
	
	condition_occurrence_id integer PRIMARY KEY,	
    person_id integer NOT NULL,				
    condition_start_date date NOT NULL, 
    condition_source_value varchar(50) NOT NULL,


In [9]:
import psycopg2
from psycopg2.extras import execute_values
import pandas as pd
import random


diagnosticos = [
    "Hipertensão arterial", "Diabetes tipo 2", "Insuficiência cardíaca", "DPOC", "Asma",
    "Enfarte agudo do miocárdio", "AVC isquémico", "Pneumonia", "Fratura do fémur", "Cancro do pulmão",
    "Insuficiência renal crónica", "Hepatite C", "Cirrose hepática", "Septicemia", "Hipotiroidismo",
    "Doença de Alzheimer", "Parkinson", "Lombalgia crónica", "Esquizofrenia", "Transtorno bipolar",
    "Depressão major", "Anemia ferropriva", "Gastrite aguda", "Úlcera péptica", "Infecção urinária",
    "COVID-19", "Apneia do sono"
]


primeiras_datas = (
    df_cleaned.groupby("person_id")["datetime"].min().reset_index()
)

rows = []
for i, row in primeiras_datas.iterrows():
    condition_occurrence_id = i + 1
    person_id = row["person_id"]
    condition_start_date = row["datetime"].date()  
    diagnostico = random.choice(diagnosticos)

    rows.append((
        condition_occurrence_id,
        person_id,
        condition_start_date,
        diagnostico
    ))

# Inserção na base de dados
conn = psycopg2.connect(
    dbname="detectid",
    user="postgres",
    password="Goncalo123",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

query = """
INSERT INTO cdmDatabaseSchema.CONDITION_OCCURRENCE (
    condition_occurrence_id,
    person_id,
    condition_start_date,
    condition_source_value
) VALUES %s
"""

execute_values(cur, query, rows)
conn.commit()

print("Dados CONDITION_OCCURRENCE inseridos com sucesso!")

cur.close()
conn.close()


Dados CONDITION_OCCURRENCE inseridos com sucesso!


### Inserção dos Dados NOTE
Queixas de Entrada, Alergias

	note_id integer PRIMARY KEY,		-- Id
	person_id integer NOT NULL,		-- Utente
	note_text TEXT NOT NULL,		-- Texto da nota
	note_type_concept_id integer NOT NULL -- Tipo de Nota


In [10]:
import random

# Inserção na base de dados
conn = psycopg2.connect(
    dbname="detectid",
    user="postgres",
    password="Goncalo123",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

queixas = [
    "Dor abdominal", "Tosse persistente", "Febre alta", "Fadiga extrema",
    "Dificuldade respiratória", "Vómitos", "Tonturas", "Palpitações"
]

alergias = [
    "Alergia a penicilina", "Intolerância à lactose", "Alergia a frutos secos",
    "Alergia a marisco", "Alergia ao pólen", "Alergia a anti-inflamatórios"
]

note_rows = []
note_id = 1
note_type_queixa = 1  # Queixas de Entrada
note_type_alergia = 2  # Alergia


pessoas_unicas = df_cleaned["person_id"].unique()

for person_id in pessoas_unicas:
    
    note_rows.append((
        note_id,
        int(person_id), # Para não ter erro de tipo
        random.choice(queixas), #Adiciona uma queixa ao calhas das acima
        note_type_queixa
    ))
    note_id += 1

    # Nem todos têm alergias
    if random.random() < 0.5:
        note_rows.append((
            note_id,
            int(person_id), # Para não ter erro de tipo
            random.choice(alergias), #Adiciona uma alergia ao calhas das acima
            note_type_alergia
        ))
        note_id += 1

# Inserir na base de dados
note_query = """
INSERT INTO cdmDatabaseSchema.NOTE (
    note_id, person_id, note_text, note_type_concept_id
) VALUES %s
"""
execute_values(cur, note_query, note_rows)
conn.commit()

print("Notas inseridas com sucesso!")


Notas inseridas com sucesso!


### Inserção dos Dados OBSERVATION
Evento

	observation_id integer PRIMARY KEY,		-- Id
	person_id integer NOT NULL,				-- Utente
	observation_concept_id integer NOT NULL, -- Tipo de Observação
    value_as_number NUMERIC NULL,            -- 0 Nao aconteceu, 1 aconteceu
	observation_datetime timestamp NOT NULL -- Data de Observação

In [11]:
# Conectar à base de dados
conn = psycopg2.connect(
    dbname="detectid",
    user="postgres",
    password="Goncalo123",
    host="localhost",
    port="5432"
)

cur = conn.cursor()


events = {
    "DESCOMPENSAÇÃO": 1,
    "Ativação Médico": 2,
    "Aumento da Vigilância": 3,
    "Via Área Ameaçada": 4
}

event_id = 1
rows = []
for _, row in df_cleaned.iterrows():
    for field, concept_id in events.items():
        rows.append((
            event_id,
            int(row["person_id"]),
            concept_id,
            row[field],
            row["datetime"]
        ))
        event_id += 1

query = """
INSERT INTO cdmDatabaseSchema.OBSERVATION (
    observation_id, person_id, observation_concept_id, value_as_number, observation_datetime
) VALUES %s
"""


execute_values(cur, query, rows)
conn.commit()

print("Dados OBSERVATION inseridos com sucesso!")
cur.close()
conn.close()


Dados OBSERVATION inseridos com sucesso!


### Inserção dos Dados PERSON_DETAILS
Detalhes do Utente (Nao esta no modelo)

	person_id integer PRIMARY KEY,             -- person_id
	first_name varchar(100) NOT NULL,          -- Primeiro Nome
	last_name varchar(100) NOT NULL            -- Último Nome


In [12]:
import psycopg2
from psycopg2.extras import execute_values

# Nomes Aleatórios
person_details = [
    (1, "Miguel", "Almeida"),
    (2, "Ana", "Ribeiro"),
    (3, "Tiago", "Santos"),
    (4, "João", "Costa"),
    (5, "Bruno", "Carvalho"),
    (6, "Sofia", "Martins"),
    (7, "Ricardo", "Lopes"),
    (8, "Pedro", "Ferreira"),
    (9, "Carlos", "Rocha"),
    (10, "André", "Nunes"),
    (11, "Diogo", "Faria"),
    (12, "Hugo", "Fonseca"),
    (13, "Rui", "Matos"),
    (14, "Mariana", "Neves"),
    (15, "Inês", "Barros"),
    (16, "Beatriz", "Mota"),
    (17, "Fábio", "Tavares"),
    (18, "Gonçalo", "Pires"),
    (19, "Marco", "Antunes"),
    (20, "Paulo", "Batista"),
    (21, "Cláudia", "Teixeira"),
    (22, "Vítor", "Mendonça"),
    (23, "Daniel", "Loureiro"),
    (24, "Raquel", "Lima"),
    (25, "Carolina", "Sousa"),
    (26, "Luís", "Borges"),
    (27, "Alexandre", "Pinto")
]

# Estabelecer a conexão com o PostgreSQL
conn = psycopg2.connect(
    dbname="detectid",
    user="postgres",
    password="Goncalo123",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

# Query de insert para PERSON_DETAILS
query_details = """
INSERT INTO cdmDatabaseSchema.PERSON_DETAILS (
    person_id, first_name, last_name
) VALUES %s
"""

# Utiliza execute_values para inserir todas as linhas de uma vez
execute_values(cur, query_details, person_details)
conn.commit()

print("Dados PERSON_DETAILS inseridos com sucesso!")

cur.close()
conn.close()



Dados PERSON_DETAILS inseridos com sucesso!


### Inserção dos Dados VISIT_OCCURENCE
Hora de Internamento, Alta, Serviço 

	visit_occurrence_id integer PRIMARY KEY,	-- Id
	person_id integer NOT NULL,			-- Utente
	care_site_id integer NULL, 			-- Serviço    1 -> Urgência, 2-> Internamento, 3-> UCI
	visit_start_datetime timestamp NOT NULL, -- Hora e Data de Internamento
	visit_end_datetime timestamp NULL -- Hora e Data de Alta

In [13]:
# Encontrar o índice da primeira medição por pessoa
idx_primeiras = df_cleaned.groupby("person_id")["datetime"].idxmin()

# Usar esses índices para obter as linhas completas (incluindo "Serviço")
primeiras_datas = df_cleaned.loc[idx_primeiras, ["person_id", "datetime", "Serviço"]].reset_index(drop=True)

rows = []
for i, row in primeiras_datas.iterrows():
    visit_occurrence_id = i + 1
    person_id = row["person_id"]
    care_site_id = row["Serviço"]
    visit_start_datetime = row["datetime"] 

    rows.append((
        visit_occurrence_id,
        person_id,
        care_site_id,
        visit_start_datetime
    ))

# Inserção na base de dados
conn = psycopg2.connect(
    dbname="detectid",
    user="postgres",
    password="Goncalo123",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

query = """
INSERT INTO cdmDatabaseSchema.VISIT_OCCURRENCE (
    visit_occurrence_id,
    person_id,
    care_site_id,
    visit_start_datetime
) VALUES %s
"""

execute_values(cur, query, rows)
conn.commit()

print("Dados VISIT_OCCURRENCE inseridos com sucesso!")

cur.close()
conn.close()

Dados VISIT_OCCURRENCE inseridos com sucesso!


### Extração dos Dados da BD para Dataset

In [ ]:
# Inserção na base de dados
conn = psycopg2.connect(
    dbname="detectid",
    user="postgres",
    password="Goncalo123",
    host="localhost",
    port="5432"
)


query = """
SELECT
  pd.first_name || ' ' || pd.last_name AS pessoa,
  TO_CHAR(
    TO_DATE(
      p.year_of_birth || '-' || COALESCE(p.month_of_birth, 1) || '-' || COALESCE(p.day_of_birth, 1),
      'MM-DD-YYYY'
    ),
    'MM-DD-YYYY'
  ) AS data_nascimento,
  CASE p.gender_concept_id
    WHEN 1 THEN 'Masculino'
    WHEN 0 THEN 'Feminino'
    ELSE 'Outro'
  END AS genero,
  v.care_site_id AS servico,
  DATE(m.measurement_datetime) AS dia_de_medicao,
  TO_CHAR(m.measurement_datetime, 'HH24:MI') AS hora_de_medicao,

  MAX(CASE WHEN m.measurement_concept_id = 1 THEN m.value_as_number END) AS ta_sistolica,
  MAX(CASE WHEN m.measurement_concept_id = 2 THEN m.value_as_number END) AS ta_diastolica,
  MAX(CASE WHEN m.measurement_concept_id = 3 THEN m.value_as_number END) AS temperatura,
  MAX(CASE WHEN m.measurement_concept_id = 4 THEN m.value_as_number END) AS spo2,
  MAX(CASE WHEN m.measurement_concept_id = 5 THEN m.value_as_number END) AS dor,
  MAX(CASE WHEN m.measurement_concept_id = 6 THEN m.value_as_number END) AS necessidade_o2,
  MAX(CASE WHEN m.measurement_concept_id = 7 THEN m.value_as_number END) AS frequencia_cardiaca,
  MAX(CASE WHEN m.measurement_concept_id = 8 THEN m.value_as_number END) AS nivel_consciencia,

  MAX(CASE WHEN o.observation_concept_id = 1 THEN o.value_as_number END) AS descompensacao,
  MAX(CASE WHEN o.observation_concept_id = 2 THEN o.value_as_number END) AS ativacao_medico,
  MAX(CASE WHEN o.observation_concept_id = 3 THEN o.value_as_number END) AS aumento_vigilancia,
  MAX(CASE WHEN o.observation_concept_id = 4 THEN o.value_as_number END) AS via_aerea_ameacada

FROM cdmDatabaseSchema.MEASUREMENT m
JOIN cdmDatabaseSchema.PERSON p ON p.person_id = m.person_id
JOIN cdmDatabaseSchema.PERSON_DETAILS pd ON pd.person_id = p.person_id
JOIN cdmDatabaseSchema.VISIT_OCCURRENCE v ON v.person_id = p.person_id
LEFT JOIN cdmDatabaseSchema.OBSERVATION o ON o.person_id = p.person_id
    AND DATE(o.observation_datetime) = DATE(m.measurement_datetime)
    AND EXTRACT(HOUR FROM o.observation_datetime) = EXTRACT(HOUR FROM m.measurement_datetime)
    AND EXTRACT(MINUTE FROM o.observation_datetime) = EXTRACT(MINUTE FROM m.measurement_datetime)

GROUP BY
  pd.first_name, pd.last_name,
  p.year_of_birth, p.month_of_birth, p.day_of_birth,
  p.gender_concept_id,
  v.care_site_id,
  DATE(m.measurement_datetime),
  TO_CHAR(m.measurement_datetime, 'HH24:MI')
ORDER BY
  pessoa, dia_de_medicao, hora_de_medicao;
"""


df = pd.read_sql(query, conn)
df.to_csv("exportacao_dados_clinicos.csv", index=False)


conn.close()

print("Exportação completa para 'exportacao_dados_clinicos.csv'")


Exportação completa para 'exportacao_dados_clinicos.csv'


C:\Users\Goncalo\AppData\Local\Temp\ipykernel_23076\601297849.py:65: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
